In [21]:
import pandas
import os
import json
import numpy as np
df = pandas.read_json('./data/logs.json')


In [22]:
df2 = df.copy()
df2['i'] = np.arange(len(df2))
df3 = df2.copy()
df2 = df2.explode('sentences')
df2['words'] = df2['sentences'].map(lambda x: x['words'])
df2['signals'] = df2['sentences'].map(lambda x: [0 if i in x['unknownIndice'] else 1 for i in range(len(x['words']))])
df3['words'] = df2.groupby('i')['words'].apply(sum)
df3['signals'] = df2.groupby('i')['signals'].apply(sum)
del df3['i']

In [26]:
df3 = df3.loc[~df3['classId'].isin(['test', 'test2'])]
userDf = df3[['userId', 'eltime', 'scorePerc']].groupby('userId').agg('mean')
userDf['eltime'] /= 1000
whiteUserDf = userDf.loc[userDf['eltime'] > 11].reset_index()
df3 = df3.merge(whiteUserDf[['userId']], on='userId', how='inner')

In [32]:
whiteUserDf = whiteUserDf.merge(df3.groupby('userId')['userId'].agg('count').to_frame('c'), on='userId', how='inner')

In [33]:
whiteUserDf

,userId,eltime,scorePerc,c
0,0bHdAlU0BUXj4OLRPzTZ,63.706667,0.454369,30
1,1yz9QcMjyGFMMdC6LPlJ,53.804545,0.033136,22
2,3dkoVNdFAoch4sxXEtWo,26.308000,-0.705699,100
3,7ksTSrU6NjGGoZqnu7uG,111.967742,0.180287,31
4,AWJ3AzY0AdAUKDqerSwH,28.882249,-0.821458,169
5,GDQqFBeAkT3BM8LaUf4e,35.259441,-0.449977,143
6,Jj9P1sIqjLLF0wVqDFeX,57.708163,-0.159752,147
7,NdlYszQsqy4y8pKYKj8N,89.949123,-1.780588,57
8,OQJ8K3jcLOM3eYaImkyN,87.883065,-1.682042,124
9,VbFRYXiXVyBLWzGDlqqm,22.784884,-0.299999,86


In [25]:
df3.to_csv('logs.csv', index=False)